In [ ]:
# --- Unique identifier (key) finder ---
st.subheader("Unique identifier (key) finder")
st.markdown(
    "We first check for a **single column** that uniquely identifies each row (no blanks, all values different). "
    "If none, you can **try combinations** (pairs/triads) that together make each row unique."
)
# Single-column keys
single_keys = card[(card["Unique"] == n_rows) & (card["Missing"] == 0)]["Column"].tolist()
if single_keys:
    st.success(f"Single-column key candidates: {', '.join(single_keys)}")
else:
    st.info("No perfect single-column key found.")

# Composite keys (guided)
with st.expander("Find composite keys (pairs/triads) — guided search"):
    st.markdown(
        "💡 **Tip:** Start small (top 6–10 columns by uniqueness). "
        "Pairs are fast to test; triads can work if pairs don’t."
    )
    max_cols_to_check = st.slider("Max columns considered (by uniqueness)", 3, min(12, len(df.columns)), 8)
    max_comb_size = st.select_slider("Max combination size", options=[2, 3], value=2,
                                     help="Try pairs first; if none found, try triads.")
    # choose top-N by uniqueness
    top_cols = card.sort_values("Unique", ascending=False, ignore_index=True)["Column"].head(max_cols_to_check).tolist()

    composite_hits = []
    for r in range(2, max_comb_size + 1):
        for combo in itertools.combinations(top_cols, r):
            # skip combos with any missing values
            if any(df[c].isna().any() for c in combo):
                continue
            # unique rows when using those columns together?
            if df[list(combo)].drop_duplicates().shape[0] == n_rows:
                composite_hits.append(combo)

    if composite_hits:
        st.success("Composite key candidates found:")
        for combo in composite_hits[:10]:  # keep UI tidy
            st.write(" • " + ", ".join(combo))
        if len(composite_hits) > 10:
            st.caption(f"...and {len(composite_hits) - 10} more.")
    else:
        st.warning("No composite key found within the chosen limits. Try more columns or include triads.")

st.caption("💡 Tip: Keys are useful for joining tables, tracking records, and de-duplicating.")